# MLB-Predict

### In this notebook, I will...
* import code from data.py
* instantiate an intances of my class 
* collect a range of historical game data from that team
* save the historical data to an .xlsx file
* load the historical data into this notebook
* prepare the data for training (i.e. strip unnecessary features)
* divide data into testing and training sets
* build a deep learning model using the keras library
* train the model with my data and tune hyperparameters (save model weights)
* graph performance of the model on training set
* graph performance of the model on testing set
* make predictions!

#### *Note for viewers*
There are some lines which I commented out and then re-executed as to remove large chunks of non-hideable code for when viewing on github.com. These are all noted, but generally occur during training of the model (hides excessive prints of training status during each iteration) and when collecting data (hides per game print statement indicating successful collection)

In [17]:
from data import LeagueStats, TeamStats

mlb = LeagueStats()

nym = TeamStats("New York Mets")

## Mets 2017-2023 Data retrieval
This is where I fetch the data that I will use to train my "mets-specific" model: only uses Mets games. 

Get data from each season individually as to avoid lengthy pieces of uninterrupted code execution that is vulnerable to crashing due to API request timeouts and other misc. errors. 

#### *Note*
Commented out and re-executed to remove large chunks of non-hideable code for when viewing on github.com

In [8]:
# nym_2017 = NYM.get_data(start_date="01/01/2017", end_date="12/31/2017", file_path="data/mets/seasons/2017.xlsx")

In [9]:
# nym_2018 = NYM.get_data(start_date="01/01/2018", end_date="12/31/2018", file_path="data/mets/seasons/2018.xlsx")

In [10]:
# nym_2019 = NYM.get_data(start_date="01/01/2019", end_date="12/31/2019", file_path="data/mets/seasons/2019.xlsx")

In [11]:
# nym_2020 = NYM.get_data(start_date="01/01/2020", end_date="12/31/2020", file_path="data/mets/seasons/2020.xlsx")

In [12]:
# nym_2021= NYM.get_data(start_date="01/01/2021", end_date="12/31/2021", file_path="data/mets/seasons/2021.xlsx")

In [13]:
# nym_2022 = NYM.get_data(start_date="01/01/2022", end_date="12/31/2022", file_path="data/mets/seasons/2022.xlsx")

In [14]:
# nym_2023 = NYM.get_data(start_date="01/01/2023", end_date="07/01/2023", file_path="data/mets/seasons/2023.xlsx")

## MLB 2023 Season Data retrieval

This is where I fetch the data that I will use to train my "2023-season-specific" model: uses all MLB games from 2023 season (up-to cutoff)

#### *Note*
Commented out and re-executed to remove large chunks of non-hideable code for when viewing on github.com

In [1]:
# mlb.get_data(start_date="04/01/2023", end_date="04/15/2023", file_path="data/seasons/2023/april_1.xlsx")

In [2]:
# mlb.get_data(start_date="04/15/2023", end_date="04/30/2023", file_path="data/seasons/2023/april_2.xlsx")

In [3]:
# mlb.get_data(start_date="05/01/2023", end_date="05/15/2023", file_path="data/seasons/2023/may_1.xlsx")

In [4]:
# mlb.get_data(start_date="05/15/2023", end_date="05/31/2023", file_path="data/seasons/2023/may_2.xlsx")

In [5]:
# mlb.get_data(start_date="06/01/2023", end_date="06/15/2023", file_path="data/seasons/2023/june_1.xlsx")

In [6]:
# mlb.get_data(start_date="06/15/2023", end_date="06/30/2023", file_path="data/seasons/2023/june_2.xlsx")

In [7]:
# mlb.get_data(start_date="07/01/2023", end_date="07/09/2023", file_path="data/seasons/2023/july_1.xlsx")

## Merging and loading data
Load and merge season data from each xlsx file into a single xlsx file

***Mets***

In [3]:
import pandas as pd
import os

# path to the individual mets seasons data sheets
mets_directory = 'data/mets/seasons'

mets_data = pd.DataFrame()

# iterate through mets data directory to retrieve each file
for filename in os.listdir(mets_directory):
    if filename.endswith('.xlsx'):
        path = os.path.join(mets_directory, filename)
        df = pd.read_excel(path)
        mets_data = pd.concat([mets_data, df], ignore_index=True)
        
# save the merged data to a new xlsx file
mets_data.to_excel('data/mets/2017-2023.xlsx')

***MLB***

In [4]:
import pandas as pd
import os

# path to individual month data sheets in 2023 season
mlb_directory = 'data/seasons/2023'

mlb_data = pd.DataFrame()

# iterate through mlb data directory to retrieve each file
for filename in os.listdir(mlb_directory):
    if filename.endswith('.xlsx'):
        path = os.path.join(mlb_directory, filename)
        df = pd.read_excel(path)
        mlb_data = pd.concat([mlb_data, df], ignore_index=True)
        
mlb_data.to_excel('data/seasons/2023.xlsx')

#### *Loading and processing data* 
Load data from the master .xlsx file into a data frame. Prepare data for training

***Mets***

In [5]:
mets_data = pd.read_excel('data/mets/2017-2023.xlsx')

# remove the game-id, date, home/away team features
mets_data.drop(columns=['game-id', 'date', 'home-team', 'away-team'], inplace=True)

# drops rows with missing labels
mets_data = mets_data.dropna(subset=['did-home-win'])

# convert 'did-home-win' labels to binary values
mets_data['did-home-win'] = mets_data['did-home-win'].astype(int)

***MLB***

In [6]:
mlb_data = pd.read_excel('data/seasons/2023.xlsx')

# remove the game-id, date, home/away team features
mlb_data.drop(columns=['game-id', 'date', 'home-team', 'away-team'], inplace=True)

# drops rows with missing labels
mlb_data = mlb_data.dropna(subset=['did-home-win'])

# convert 'did-home-win' labels to binary values
mlb_data['did-home-win'] = mlb_data['did-home-win'].astype(int)

## Data Processing
#### *Experminetal Optimization*
Rearrange the order of the features to attempt to optimize the model

* Order 1: Place most important features first with each home team statistic immediately followed by the away's counter part. Allows for many meaningful comparisons between adjacent features

In [3]:
order1 = ['did-home-win',
          'home-win-percentage', 'away-win-percentage',
          'home-starter-season-era', 'away-starter-season-era',
          'home-elo-probability', 'away-elo-probability', 
          'home-rating-probability', 'away-rating-probability',
          'home-starter-season-win-percentage', 'away-starter-season-win-percentage',
          'home-last10-avg-runs', 'away-last10-avg-runs',
          'home-last10-avg-ops', 'away-last10-avg-ops',
          'home-last10-avg-runs-allowed', 'away-last10-avg-runs-allowed', 
          'home-starter-season-avg', 'away-starter-season-avg',
          'home-elo-pregame', 'away-elo-pregame',
          'home-pitcher-rgs', 'away-pitcher-rgs',
          'home-last10-avg-hits', 'away-last10-avg-hits',
          'home-last10-avg-hits-allowed', 'away-last10-avg-hits-allowed',
          'home-last10-avg-obp', 'away-last10-avg-obp',
          'home-rating-pregame', 'away-rating-pregame',
          'home-starter-season-runs-per9', 'away-starter-season-runs-per9',
          'home-last10-avg-strikeouts', 'away-last10-avg-strikeouts',
          'home-starter-career-era', 'away-starter-career-era']

order2 = ['did-home-win',
          'home-win-percentage', 'home-starter-season-era', 
          'home-elo-probability', 'home-rating-probability', 
          'home-starter-season-win-percentage', 'home-last10-avg-runs', 
          'home-last10-avg-ops', 'home-last10-avg-runs-allowed', 
          'home-starter-season-avg', 'home-elo-pregame', 
          'home-pitcher-rgs', 'home-last10-avg-hits', 
          'home-last10-avg-hits-allowed', 'home-last10-avg-obp', 
          'home-rating-pregame', 'home-starter-season-runs-per9', 
          'home-last10-avg-strikeouts', 'home-starter-career-era', 
          'away-win-percentage', 'away-starter-season-era', 
          'away-elo-probability', 'away-rating-probability',
          'away-starter-season-win-percentage', 'away-last10-avg-runs',
          'away-last10-avg-ops','away-last10-avg-runs-allowed', 
          'away-starter-season-avg', 'away-elo-pregame',
          'away-pitcher-rgs', 'away-last10-avg-hits',
          'away-last10-avg-hits-allowed', 'away-last10-avg-obp',
          'away-rating-pregame', 'away-starter-season-runs-per9',
          'away-last10-avg-strikeouts', 'away-starter-career-era']

***Mets***

In [8]:
# reorder the columns
mets_data = mets_data[order1]

***MLB***

In [9]:
# reorder the columns
mlb_data = mlb_data[order1]

#### *Drop rows with missing labels*
Drops all rows that are missing more than THRESHOLD features

In [10]:
# constant representing the amount of features that must missing for a row to be excluded/removed
THRESHOLD = 10

***Mets***

In [11]:
# drops rows with too many missing features
mets_data = mets_data.dropna(thresh=df.shape[1] - THRESHOLD)

***MLB***

In [12]:
# drops rows with too many missing features
mlb_data = mlb_data.dropna(thresh=df.shape[1] - THRESHOLD)

#### *Min-Max Feature Normalization*
Normalize the numeric features to a scale of [0,1]

In [13]:
from sklearn.preprocessing import MinMaxScaler

mets_scaler = MinMaxScaler()
mlb_scaler  = MinMaxScaler()

# scale all columns that don't represent averages between 0-1 (i.e. batting avg doesn't need scaling)
columns_to_scale = ['home-starter-season-era', 'away-starter-season-era',
                   'home-last10-avg-runs', 'away-last10-avg-runs',
                   'home-last10-avg-runs-allowed', 'away-last10-avg-runs-allowed', 
                   'home-elo-pregame', 'away-elo-pregame',
                   'home-pitcher-rgs', 'away-pitcher-rgs',
                   'home-last10-avg-hits', 'away-last10-avg-hits',
                   'home-last10-avg-hits-allowed', 'away-last10-avg-hits-allowed',
                   'home-rating-pregame', 'away-rating-pregame',
                   'home-starter-season-runs-per9', 'away-starter-season-runs-per9',
                   'home-last10-avg-strikeouts', 'away-last10-avg-strikeouts',
                   'home-starter-career-era', 'away-starter-career-era']

***Mets***

In [16]:
import pickle

# apply min-max normalization to selected features
mets_data[columns_to_scale] = mets_scaler.fit_transform(mets_data[columns_to_scale])

scaler_path = 'models/scalers/'
with open(scaler_path + 'mets6year_scaler.pkl', 'wb') as file:
    pickle.dump(mets_scaler, file)

***MLB***

In [17]:
import pickle

# apply min-max normalization to selected features
mlb_data[columns_to_scale] = mlb_scaler.fit_transform(mlb_data[columns_to_scale])

scaler_path = 'models/scalers/'
with open(scaler_path + 'mlb2023_scaler.pkl', 'wb') as file:
    pickle.dump(mlb_scaler, file)

#### *Randomize data order, Split training and testing data*
Ensures that training and testing data aren't chronologically grouped. Thus, will treat each game more independently

***Mets***

In [18]:
import numpy as np

# randomize the order of the rows in the dataframe
mets_data = mets_data.sample(frac=1).reset_index(drop=True)

# separate labels from the features
mets_features = mets_data.drop('did-home-win', axis=1).values
mets_labels = mets_data['did-home-win'].values

# verify shapes features and labels
print("Mets features shape: ", mets_features.shape)
print("Mets labels shape: ", mets_labels.shape)

mets_indices = list(range(len(mets_features)))
mets_split_index = int(0.85 * len(mets_features))

mets_train_indices = mets_indices[:mets_split_index]
mets_test_indices  = mets_indices[mets_split_index:]

mets_x_train = mets_features[mets_train_indices]
mets_x_test  = mets_features[mets_test_indices ]
mets_y_train = mets_labels[mets_train_indices]
mets_y_test  = mets_labels[mets_test_indices ]

# verify shapes of training/testing sets
print("Mets training set shape: ", mets_x_train.shape, mets_y_train.shape)
print("Mets testing set shape: " , mets_x_test.shape,  mets_y_test.shape )

Mets features shape:  (899, 36)
Mets labels shape:  (899,)
Mets training set shape:  (764, 36) (764,)
Mets testing set shape:  (135, 36) (135,)


***MLB***

In [19]:
import numpy as np

# randomize the order of the rows in the dataframe
mlb_data = mlb_data.sample(frac=1).reset_index(drop=True)

# separate labels from the features
mlb_features = mlb_data.drop('did-home-win', axis=1).values
mlb_labels = mlb_data['did-home-win'].values

# verify shapes features and labels
print("MLB features shape: ", mlb_features.shape)
print("MLB labels shape: ", mlb_labels.shape)

mlb_indices = list(range(len(mlb_features)))
mlb_split_index = int(0.85 * len(mlb_features))

mlb_train_indices = mlb_indices[:mlb_split_index]
mlb_test_indices  = mlb_indices[mlb_split_index:]

mlb_x_train = mlb_features[mlb_train_indices]
mlb_x_test  = mlb_features[mlb_test_indices ]
mlb_y_train = mlb_labels[mlb_train_indices]
mlb_y_test  = mlb_labels[mlb_test_indices ]

# verify shapes of training/testing sets
print("MLB training set shape: ", mlb_x_train.shape, mlb_y_train.shape)
print("MLB testing set shape: " , mlb_x_test.shape,  mlb_y_test.shape )

MLB features shape:  (1353, 36)
MLB labels shape:  (1353,)
MLB training set shape:  (1150, 36) (1150,)
MLB testing set shape:  (203, 36) (203,)


## Building the model
Using LightGBM model

In [20]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score

# model parameters 
params = {
    'objective': 'binary',
    'metric': 'accuracy',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
}

***Mets***

In [21]:
# create a LightGBM Dataset with training features and labels
mets_train_data = lgb.Dataset(mets_x_train, label=mets_y_train)

***MLB***

In [22]:
# create a LightGBM Dataset with training features and labels
mlb_train_data = lgb.Dataset(mlb_x_train, label=mlb_y_train)

## Training the models

***Mets***

#### *Note*
Commented out and re-executed to remove large chunks of non-hideable code for when viewing on github.com

In [15]:
# mets_model = lgb.train(params, mets_train_data, num_boost_round=1000)
# mets_model.save_model('models/mets6year.txt')

***MLB***

#### *Note*
Commented out and re-executed to remove large chunks of non-hideable code for when viewing on github.com

In [16]:
# mlb_model = lgb.train(params, mlb_train_data, num_boost_round=1000)
# mlb_model.save_model('models/mlb2023.txt')

## Testing the models
Here I test the models' accuracy on their own testing sets. This tests each model individually. I.e. the mets model is tested on predicting only mets games (random scatter from 2017-2023) and the mlb model is tested on predicting any mlb game from the 2023 season (up to CUTOFF)

***Mets***

In [25]:
mets_y_pred = mets_model.predict(mets_x_test)
mets_y_pred_binary = (mets_y_pred > 0.5).astype(int)
accuracy = accuracy_score(mets_y_test, mets_y_pred_binary)

print('Accuracy:', accuracy)

Accuracy: 0.6


***MLB***

In [26]:
mlb_y_pred = mlb_model.predict(mlb_x_test)
mlb_y_pred_binary = (mlb_y_pred > 0.5).astype(int)
accuracy = accuracy_score(mlb_y_test, mlb_y_pred_binary)

print('Accuracy:', accuracy)

Accuracy: 0.6699507389162561


## Predictions!
This my first prediction, made on 07/14/2023 (morning), that suggests that the New York Mets will lose to the Los Angeles Dodgers tonight at home. 

In [1]:
from data import LeagueStats

mlb = LeagueStats()

mlb.predict_next_game("mlb2023", "New York Mets")

('Los Angeles Dodgers',
 0.002204909619759474,
 {'datetime': '2023-07-14T23:10:00Z',
  'date': '2023-07-14',
  'away': 'Los Angeles Dodgers',
  'home': 'New York Mets',
  'home_probable': 'Justin Verlander',
  'away_probable': 'Julio Urias',
  'venue': 'Citi Field',
  'national_broadcasts': ['Apple TV+'],
  'series_status': None,
  'summary': '2023-07-14 - Los Angeles Dodgers @ New York Mets (Scheduled)'})

## Initial Remarks

Of the 12 or so predictions that I have made on this first day using the mlb2023 model, I have some observations. The most noticeable of which being that the actual numeric prediction ([0,1]) tends very heavily towards the boundaries (0 and 1). Many of the predictions were well below (0.01) and some of the positive (1) predictions were above 0.98 or so. As I haven't seen any of these games play out yet and evaluated the model on real future data, I can't yet truly comment on this, but it was interesting to note that only 2 of the 12 predictions I made were truly tending towards the center (0.318 and 0.365). 

The model has also made some bold predictions, betting against the sportsbooks on more than one of these games. Of the 12 game slate for tonight (07/14/2023), the model predicted upsets including... Rockies(+180) over the Yankees(-210), Guardians(+130) over the Rangers(-150), and Nationals(+150) over the Cardinals(-180). The other picks, were either aligned with the betting favorite or were games that didn't have a discernable favorite. As in the Mets game, there isn't a clear betting favorite (note: Urias vs. Verlander), but the model quite confidently believes in the Dodgers to win it. We shall see!

## Hopefully Improved Models & Generalized Data Preparation
From here on out, I am going to simplify the data preparation for training into a function so that I can easily mess around and train new models with the goal of surpassing the ~65% testing accuracy that my "mlb2023" model is able to achieve. 

In [12]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd        
import numpy as np
import pickle
import os

def prepare_data(data_dirs, model_name, order=order2 save_dir=None, missing_data_threshold=10):
    """
    Args:
        data_dirs: list of paths to folders with the .xlsx data sheets
        save_dir: file_path to save merged .xlsx sheet to if desired
            -> must end in .xlsx and be a valid (existing) file path
    
    Returns:
        x_train, x_test, y_train, y_test
    """
    df = pd.DataFrame()
    # iterate through mlb data directory to retrieve each file
    for dir in data_dirs:
        for filename in os.listdir(dir):
            if filename.endswith('.xlsx'):
                path = os.path.join(dir, filename)
                d = pd.read_excel(path)
                df = pd.concat([df, d], ignore_index=True)
    if save_dir:
        df.to_excel(save_dir)
    
    # remove the game-id, date, home/away team features
    df.drop(columns=['game-id', 'date', 'home-team', 'away-team'], inplace=True)
    # drops rows with missing labels
    df = df.dropna(subset=['did-home-win'])
    # convert 'did-home-win' labels to binary values
    df['did-home-win'] = df['did-home-win'].astype(int)
    
    df = df[order]
    
    df = df.dropna(thresh=df.shape[1] - missing_data_threshold)
    
    scaler = MinMaxScaler()
    # scale all columns that don't represent averages between 0-1 (i.e. batting avg doesn't need scaling)
    columns_to_scale = ['home-starter-season-era', 'away-starter-season-era',
                       'home-last10-avg-runs', 'away-last10-avg-runs',
                       'home-last10-avg-runs-allowed', 'away-last10-avg-runs-allowed', 
                       'home-elo-pregame', 'away-elo-pregame',
                       'home-pitcher-rgs', 'away-pitcher-rgs',
                       'home-last10-avg-hits', 'away-last10-avg-hits',
                       'home-last10-avg-hits-allowed', 'away-last10-avg-hits-allowed',
                       'home-rating-pregame', 'away-rating-pregame',
                       'home-starter-season-runs-per9', 'away-starter-season-runs-per9',
                       'home-last10-avg-strikeouts', 'away-last10-avg-strikeouts',
                       'home-starter-career-era', 'away-starter-career-era']
    # apply min-max normalization to selected features
    df[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])
    scaler_path = 'models/scalers/'
    with open(scaler_path + model_name + '_scaler.pkl', 'wb') as file:
        pickle.dump(scaler, file)

    # randomize the order of the rows in the dataframe
    df = df.sample(frac=1).reset_index(drop=True)
    # separate labels from the features
    features = df.drop('did-home-win', axis=1).values
    labels = df['did-home-win'].values
    # verify shapes features and labels
    print("Features shape: ", df.shape)
    print("Labels shape: ", df.shape)
    indices = list(range(len(features)))
    split_index = int(0.85 * len(features))
    train_indices = indices[:split_index]
    test_indices  = indices[split_index:]
    x_train = features[train_indices]
    x_test  = features[test_indices ]
    y_train = labels[train_indices]
    y_test  = labels[test_indices ]
    # verify shapes of training/testing sets
    print("Training set shape: ", x_train.shape, y_train.shape)
    print("Testing set shape: " , x_test.shape,  y_test.shape )
    return x_train, x_test, y_train, y_test

## MLB 3 Year
Using the data_retrieval.py script, I collected game data from 2021-2023 seasons in the background and I'm going to pull that data and use it to train a model. The data is from every MLB game during these 3 season (2023 cutoff is 07/09). 

In [32]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score

data = ['data/seasons/2021', 'data/seasons/2022', 'data/seasons/2023']
x_train, x_test, y_train, y_test = prepare_data(data_dirs=data, model_name="mlb3year", order=order2, missing_data_threshold=10)

# model parameters 
params = {
    'objective': 'binary',
    'metric': 'accuracy',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
}

data = lgb.Dataset(x_train, label=y_train)

model = lgb.train(params, data, num_boost_round=1000)
model.save_model('models/mlb3year.txt')

y_pred = model.predict(x_test)
y_pred_binary = (y_pred > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred_binary)

print('Accuracy:', accuracy)

Features shape:  (5975, 37)
Labels shape:  (5975, 37)
Training set shape:  (5078, 36) (5078,)
Testing set shape:  (897, 36) (897,)
[LightGBM] [Info] Number of positive: 2709, number of negative: 2369
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000532 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8532
[LightGBM] [Info] Number of data points in the train set: 5078, number of used features: 36
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.533478 -> initscore=0.134112
[LightGBM] [Info] Start training from score 0.134112
Accuracy: 0.6376811594202898


## Testing different feature orders
In the past 3 models, I've used "order1" which creates direct comparisons between adjacent features by placing a home team's statistic directly next to the away team's same stat. Here I will try a different order, primarily the order where I place all of one team's statistics first and then the other team's after. Below I will use the same data as ***mlb3year***

In [18]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score

data = ['data/seasons/2021', 'data/seasons/2022', 'data/seasons/2023']
x_train, x_test, y_train, y_test = prepare_data(data_dirs=data, model_name="mlb3year_test", order=order2, missing_data_threshold=10)

# model parameters 
params = {
    'objective': 'binary',
    'metric': 'accuracy',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
}

data = lgb.Dataset(x_train, label=y_train)

model = lgb.train(params, data, num_boost_round=1000)
model.save_model('models/mlb3year_test.txt')

y_pred = model.predict(x_test)
y_pred_binary = (y_pred > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred_binary)

print('Accuracy:', accuracy)

Features shape:  (5975, 37)
Labels shape:  (5975, 37)
Training set shape:  (5078, 36) (5078,)
Testing set shape:  (897, 36) (897,)
[LightGBM] [Info] Number of positive: 2727, number of negative: 2351
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000804 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8551
[LightGBM] [Info] Number of data points in the train set: 5078, number of used features: 36
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.537022 -> initscore=0.148361
[LightGBM] [Info] Start training from score 0.148361
Accuracy: 0.6432552954292085


### Remarks
After training the model a couple times with "order2", it appears that the order doesn't make a significant impact on the model's testing accuracy. If anything, however, I've noticed that the training accuracies were on average a little bit higher. Moving forward, I will use order2 as the default feature order. 

Additionally, I will retrain the official mlb3year model with order2, so that the saved model is up to date.  